## One-Way Model

Consider the following data from a hypothetical one-way experiment with four levels of a treatment.

### Data

In [2]:
using DataFrames

In [3]:
df = readtable("../data/OneWay.dat", separator = ' ')

,A,y
1,1,1.1
2,1,1.2
3,2,1.9
4,2,1.2
5,2,2.0
6,2,1.7
7,3,1.0
8,3,1.7
9,4,1.1
10,1,1.7


The $\mathbf{X}$ matrix for the one way model

$$
y_{ij} = \mu + \alpha_i + e_{ij}
$$

is

$$
\mathbf{X} = 
\begin{bmatrix}
1 & 1 & 0 & 0 & 0 \\
1 & 1 & 0 & 0 & 0 \\
1 & 0 & 1 & 0 & 0 \\
1 & 0 & 1 & 0 & 0 \\
1 & 0 & 1 & 0 & 0 \\
1 & 0 & 1 & 0 & 0 \\
1 & 0 & 0 & 1 & 0 \\
1 & 0 & 0 & 1 & 0 \\
1 & 0 & 0 & 0 & 1 \\
1 & 1 & 0 & 0 & 0 \\
\end{bmatrix}
$$


Note that any row of $\mathbf{X}$ contains only two non-zero elements. These correspond to $\mu$ and $\alpha_i$ in the model. Recall that $\mathbf{x}_i$ denotes row $i$ of $\mathbf{X}$. The first element of $\mathbf{x}_i$ corresponds to $\mu$. Thus, all $\mathbf{x}_i$ will contain a "1" in this position. The second element of $\mathbf{x}_i$ corresponds to $\alpha_1$. Thus, $\mathbf{x}_1$, $\mathbf{x}_2$ and $\mathbf{x}_{10}$ contain a "1" in this position because observations 1, 2 and 10 are from treatment 1. So, the contribution from the first observation, for example,  to the $\mathbf{X'X}$ matrix is

$$
\mathbf{x}_1\mathbf{x}'_1 = 
\begin{bmatrix}
1 \\
1 \\
0 \\
0 \\
0
\end{bmatrix} 
\begin{bmatrix}
1 & 1 & 0 & 0 & 0 
\end{bmatrix} = 
\begin{bmatrix}
1 & 1 & 0 & 0 & 0 \\
1 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 
\end{bmatrix}
$$

The contributions from observations 2 and 10 would be identical to this because $\mathbf{x}_2$ and $\mathbf{x}_{10}$ are identical to $\mathbf{x}_1$. In general, set posmu = 1, which is the column position of the "1" in any $\mathbf{x}_i'$ corresponding to μ, and set posA equal to the column position for the "1" in any $\mathbf{x}_i'$ corresponding to $α_i$. Then, the positions of the contributions to $\mathbf{X'X}$ from any observations are: (posmu,posmu), (posmu,posA), (posA,posmu) and (posA,posA). Further, in the one-way model, the contribution to each of these positions is a "1". So, $\mathbf{X'X}$ can be constructed efficiently by setting postrt = 1 + A, where A is the level of the factor A for observation $i$, and adding "1" to positions (posmu,posmu), (posmu,posA), (posA,posmu) and (posA,posA) in $\mathbf{X'X}$ for each observation in the data file. Similarly, $\mathbf{X'y}$ can be constructed efficiently by adding $\mathbf{y}_i$ to positions posmu and posA in $\mathbf{X'y}$. This strategy is used in the program given below.

#### Levels in A

In [4]:
unique(df[:A])

4-element DataArrays.DataArray{Int64,1}:
 1
 2
 3
 4

#### Number of levels in A 

In [5]:
levelsA = length(unique(df[:A]))

4

#### Make X'X and X'y

In [6]:
p = levelsA + 1
XPX = fill(0.0,p,p)
xpy = fill(0.0,p);
for i in 1:size(df,1)
    posMu = 1
    posA  = 1 + df[i,:A]
    y     = df[i,:y]
    XPX[posMu,posMu] += 1.0
    XPX[posMu,posA]  += 1.0
    XPX[posA,posMu]  += 1.0
    XPX[posA,posA]   += 1.0
    xpy[posMu]  += y
    xpy[posA]   += y   
end

In [7]:
XPX

5x5 Array{Float64,2}:
 10.0  3.0  4.0  2.0  1.0
  3.0  3.0  0.0  0.0  0.0
  4.0  0.0  4.0  0.0  0.0
  2.0  0.0  0.0  2.0  0.0
  1.0  0.0  0.0  0.0  1.0

In [8]:
xpy

5-element Array{Float64,1}:
 14.6
  4.0
  6.8
  2.7
  1.1

#### Solution
Note $\mathbf{X'X}$ is singular, but a solution can be obtained as follows. 

In [9]:
sol = XPX\xpy

5-element Array{Float64,1}:
  1.74583  
 -0.4125   
 -0.0458333
 -0.395833 
 -0.645833 

#### Verify solution 

In [10]:
[XPX*sol xpy]

5x2 Array{Float64,2}:
 14.6  14.6
  4.0   4.0
  6.8   6.8
  2.7   2.7
  1.1   1.1

### comparison of efficiencies of different ways to calculate X'X

### Big Example

In [2]:
using Distributions

#### Generate data

In [3]:
n = 1000000
p = 1000
A = sample([1:p],n)
α = randn(p)
y = [α[i] + randn(1) for i in A];

### Computing X'X as product of full-stored (X' and X)

In [16]:
X = fill(0.0,(n,p));

In [18]:
@time for i = 1:n
    j = A[i]
    X[i,j] = 1.0
end

elapsed time: 0.374754528 seconds (71887704 bytes allocated, 33.93% gc time)


In [20]:
@time XPX = X'X;

elapsed time: 12.361388595 seconds (8000192 bytes allocated)


### Computing full-stored X'X without matrix multiplication

In [21]:
XPX = fill(0.0,p,p);

In [23]:
@time for i in 1:size(A,1)
    posA  = A[i]
    XPX[posA,posA]   += 1.0
end

elapsed time: 0.509628998 seconds (103798408 bytes allocated, 40.25% gc time)


In [24]:
12.36/.51

24.235294117647058

### Computing sparse-stored X'X as product of sparse-stored (X' and X) <font color="red">best</font>

In [8]:
ii = 1:n
@time X = sparse(ii,A,1.0);

elapsed time: 0.115371995 seconds (64016736 bytes allocated, 51.93% gc time)


In [31]:
@time XPX = X'X;

elapsed time: 0.242494377 seconds (56073456 bytes allocated)


### Computing sparse-stored X'X without matrix multiplication

In [33]:
XPX = spzeros(p,p)

1000x1000 sparse matrix with 0 Float64 entries:

In [36]:
@time for i in 1:size(A,1)
    posA  = A[i]
    XPX[posA,posA]   += 1.0
end

elapsed time: 0.533198848 seconds (103798376 bytes allocated, 38.87% gc time)
